In [1]:
# 데이터 붙이기 => 번호 컬럼만 따로 가져오기 => 기존 데이터에 내용 추가해보기

In [180]:
import numpy as np
import pandas as pd
import requests 
import time
from bs4 import BeautifulSoup 
from glob import glob

file_names = glob("/Users/ppangppang/Downloads/*.csv")
file_names

total = pd.DataFrame()
for file_name in file_names:
    temp = pd.read_csv(file_name)
    total = pd.concat([total, temp])

In [200]:
# 사용할 데이터만가져오기, 기존 데이터에 있는 종가는 현재 가격이 아님
total.index = range(len(total))
df = total[['종목','기호','거래량']]
df

,종목,기호,거래량
0,에스디바이오센서,137310,936.74K
1,이마트,139480,176.55K
2,메리츠금융지주,138040,258.07K
3,엘엔에프,066970,0.00
4,현대중공업지주,267250,913.40K
...,...,...,...
545,하나금융지주,086790,879.20K
546,포스코케미칼,003670,425.56K
547,엔씨소프트,036570,194.65K
548,아모레퍼시픽,090430,402.65K


In [201]:
code = df['기호']
len(code)

550

In [198]:
# code= df['기호'][0:3]
# code[0]='000000'# 오류테스트용
# code

In [161]:
# 종목번호를 잘못입력해도 연결되고, html 변환도 오류 안뜬다

# url = f'https://finance.naver.com/item/main.nhn?code=000000'
# response = requests.get(url)
# response
# html = BeautifulSoup(response.text, 'html.parser')

In [162]:
# 오류 뜨기전 사용하던 코드
# price = []

# for num in code:
#     url = f'https://finance.naver.com/item/main.nhn?code={num}'

#     print(f'{num}종목 현재가격 크롤링중 입니다.')
#     seed = np.random.randint(100) # seed 설정
#     np.random.seed(seed)    # 시드 생성
#     a = np.random.randint(5)  # 시드에서 난수 생성
#     time.sleep(a)     # 코드실행 지연시키는 코드
    
#     response = requests.get(url)

#     if(response.status_code == requests.codes.ok):
#         html = BeautifulSoup(response.text, 'html.parser')

#     else:
# #         print(f'{num}종목 크롤링 실패, 값을 다시 확인해주세요(금일종가 0원으로 처리)')
#         print(f"크롤링 실패. 종목번호 : {num} 을 확인해 주세요")
# #         price.append('0')
# #         continue

#     price.append(html.select('p.no_today')[0].text)


In [204]:
# 오류 테스트용
price = []

for num in code:
    url = f'https://finance.naver.com/item/main.nhn?code={num}'

    print(f'{num}종목 현재가격 크롤링중 입니다.')
    seed = np.random.randint(100) # seed 설정
    np.random.seed(seed)    # 시드 생성
    a = np.random.randint(5)  # 시드에서 난수 생성
    time.sleep(a)     # 코드실행 지연시키는 코드
    
    response = requests.get(url)

    if(response.status_code == requests.codes.ok):
        html = BeautifulSoup(response.text, 'html.parser')
           
    else:
        print(f"크롤링 실패. 종목번호 : {num} 을 확인해 주세요")

    try:
        price.append(html.select('p.no_today')[0].text)

    except:
        print(f'{num}종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)')
        price.append('0')


137310종목 현재가격 크롤링중 입니다.
139480종목 현재가격 크롤링중 입니다.
138040종목 현재가격 크롤링중 입니다.
066970종목 현재가격 크롤링중 입니다.


KeyboardInterrupt: 

In [208]:
# price 정리

code_price =[]

for num in price:
    temp = []
        # 현재가격 크롤링 되지 않는 데이터를 html.select형태로 변환해서 작업해버리자
    if num == '0':
        num = '\n\n0\n0\n\n'
        temp.append(num.split('\n'))
        temp=np.array(temp)
        code_price.append(temp[0,2:3])

    else:
        temp.append(num.split('\n'))
        temp=np.array(temp)
        code_price.append(temp[0,2:3])

code_price = np.ravel(code_price, order='C')
code_price = pd.DataFrame({"현재가격" : code_price})


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [206]:
# 현재 가격 열 붙이기
df_daily = df.join(code_price, how='outer')

In [207]:
# 최종확인
df_daily

,종목,기호,거래량,현재가격
0,에스디바이오센서,137310,936.74K,"48,250"
1,이마트,139480,176.55K,"174,500"
2,메리츠금융지주,138040,258.07K,"37,150"
3,엘엔에프,066970,0.00,NaN
4,현대중공업지주,267250,913.40K,NaN
...,...,...,...,...
545,하나금융지주,086790,879.20K,NaN
546,포스코케미칼,003670,425.56K,NaN
547,엔씨소프트,036570,194.65K,NaN
548,아모레퍼시픽,090430,402.65K,NaN
